In [43]:
import os
import django
os.chdir('/app')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'testproject.settings')
django.setup()

# Formkit-Ninja

What is Formkit-Ninja?
It's a Django application to store and convert FormKit schemas between JSON, Pydantic models, and database backend. It supports most of the FormKit syntax as well as some of the "extra" goodies.

In [44]:

# Here we're defining some functions to help us display data in a nice way 

from typing import Any
from rich.table import Table
from rich.jupyter import print
from pydantic import BaseModel

def kvtable(inputelement: dict[str, Any], title: str | None = None):
    table = Table(title=title)
    table.add_column("Key", style="cyan", no_wrap=True)
    table.add_column("Value", style="magenta")
    for key, value in sorted(inputelement.items()):
        table.add_row(str(key), str(value))
    print(table)

def modeltable(inputmodel: BaseModel):
    kvtable(inputmodel.model_dump(exclude_none=True, by_alias=True), title=inputmodel.__class__.__name__)

def manytable(inputelements: list[dict[str, Any]], title: str | None = None):
    table = Table(title=title)
    table.add_column("Index", style="cyan", no_wrap=True)
    # Set of all keys
    keys = set()
    for element in inputelements:
        keys.update(element.keys())
    # These are the "rows"
    rows = sorted(keys)
    # These are the "columns"
    columns = range(len(inputelements))
    # Add the columns
    for index, i in enumerate(inputelements):
        table.add_column(str(index), style="magenta")
    # Add the rows
    for row in rows:
        content = []
        for index, i in enumerate(inputelements):
            content.append(str(i.get(row, "")))
        table.add_row(row, *content)
    print(table)

def manynodetable(inputelements: list[BaseModel], title: str | None = None):
    manytable([i.model_dump(exclude_none=True, by_alias=True) for i in inputelements], title=title) 
    


In [45]:
# Add a "Telephone" input

# This is a simple input with validation and a label
# Note that the label is translated using the gettext function (client side)
tel = {
    "$formkit": "tel",
    "label": '$gettext("Phone number")',
    "maxLength": 8,
    "name": "phone_number",
    "validation": "number|length:8,8",
}
kvtable(tel)

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key        ┃ Value                    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit   │ tel                      │
│ label      │ $gettext("Phone number") │
│ maxLength  │ 8                        │
│ name       │ phone_number             │
│ validation │ number|length:8,8        │
└────────────┴──────────────────────────┘

In [46]:

# This is in Python / Pydantic a "TelNode"
# We can create this node from the dictionary above

from formkit_ninja.formkit_schema import TelNode

node = TelNode(**tel)

modeltable(node)

                 TelNode                 
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key        ┃ Value                    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit   │ tel                      │
│ label      │ $gettext("Phone number") │
│ maxLength  │ 8                        │
│ name       │ phone_number             │
│ validation │ number|length:8,8        │
└────────────┴──────────────────────────┘

In [47]:
# What if we don't know the type of node beforehand?
# We can use the "FormkitNode" class to create the node from the dictionary
from formkit_ninja.formkit_schema import DiscriminatedNodeType
node = DiscriminatedNodeType(**tel)

# The DiscriminatedNodeType can convert a variety of "node" types
modeltable(node)

          DiscriminatedNodeType          
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key        ┃ Value                    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit   │ tel                      │
│ label      │ $gettext("Phone number") │
│ maxLength  │ 8                        │
│ name       │ phone_number             │
│ validation │ number|length:8,8        │
└────────────┴──────────────────────────┘

In [48]:
node

DiscriminatedNodeType(root=TelNode(children=None, key=None, if_condition=None, for_loop=None, bind=None, meta=None, id=None, name='phone_number', label='$gettext("Phone number")', help=None, validation='number|length:8,8', validationLabel=None, validationVisibility=None, validationMessages=None, placeholder=None, value=None, prefixIcon=None, classes=None, additional_props={'maxLength': 8}, node_type='formkit', readonly=None, formkit='tel'))

In [64]:
# And, we can save this in the database...

from formkit_ninja.models import FormKitSchemaNode

# This has a `from_pydantic` method
nodes = list(FormKitSchemaNode.from_pydantic(node))


[00:48:02] Creating  ba4a5d27-a515-40cd-9e23-2a30227e8a25                                             ]8;id=861634;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=594247;file:///app/src/formkit_ninja/models.py#570\570]8;;\

           Yielding: Node: phone_number                                                               ]8;id=897297;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=119011;file:///app/src/formkit_ninja/models.py#592\592]8;;\

In [50]:
nodes

[<FormKitSchemaNode: Node: phone_number>]

In [51]:
for n in nodes:
    kvtable(n.get_node_values())
    modeltable(n.get_node())

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key              ┃ Value                    ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit         │ tel                      │
│ additional_props │ {'maxLength': 8}         │
│ label            │ $gettext("Phone number") │
│ name             │ phone_number             │
│ validation       │ number|length:8,8        │
└──────────────────┴──────────────────────────┘

                 TelNode                 
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key        ┃ Value                    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit   │ tel                      │
│ label      │ $gettext("Phone number") │
│ maxLength  │ 8                        │
│ name       │ phone_number             │
│ validation │ number|length:8,8        │
└────────────┴──────────────────────────┘

## A More Complex Example

The following example is a "datepicker"

In [52]:
sf_41_datpicker = {
    "$formkit": "datepicker",
    "_currentDate": "$getCurrentDate",
    "calendarIcon": "calendar",
    "format": "DD/MM/YYYY",
    "id": "date",
    "key": "date",
    "label": '$gettext("Date")',
    "name": "date",
    "nextIcon": "angleRight",
    "prevIcon": "angleLeft",
    "sectionsSchema": {
        "day": {
            "children": [
                "$day.getDate()",
                {
                    "children": [
                        {
                            "children": [
                                {
                                    "$el": "div",
                                    "attrs": {"class": "formkit-day-highlight"},
                                    "if": "$attrs._currentDate().year === $day.getFullYear()",
                                }
                            ],
                            "if": "$attrs._currentDate().month === $day.getMonth()",
                        }
                    ],
                    "if": "$attrs._currentDate().day === $day.getDate()",
                },
            ]
        }
    },
}


In [53]:
# When we run this we should get a valid node
sf_41_node = DiscriminatedNodeType(**sf_41_datpicker)

In [54]:
kvtable(sf_41_datpicker)
modeltable(sf_41_node)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key            ┃ Value                                                                                          ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit       │ datepicker                                                                                     │
│ _currentDate   │ $getCurrentDate                                                                                │
│ calendarIcon   │ calendar                                                                                       │
│ format         │ DD/MM/YYYY                                                                                     │
│ id             │ date                                                                                           │
│ key            │ date                                                                                           │
│ label          │ $gettext("Date")                                                                               │
│ name           │ date                                                                                           │
│ nextIcon       │ angleRight                                                                                     │
│ prevIcon       │ angleLeft                                                                                      │
│ sectionsSchema │ {'day': {'children': ['$day.getDate()', {'children': [{'children': [{'$el': 'div', 'attrs':    │
│                │ {'class': 'formkit-day-highlight'}, 'if': '$attrs._currentDate().year ===                      │
│                │ $day.getFullYear()'}], 'if': '$attrs._currentDate().month === $day.getMonth()'}], 'if':        │
│                │ '$attrs._currentDate().day === $day.getDate()'}]}}                                             │
└────────────────┴────────────────────────────────────────────────────────────────────────────────────────────────┘

                                               DiscriminatedNodeType                                               
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key            ┃ Value                                                                                          ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit       │ datepicker                                                                                     │
│ _currentDate   │ $getCurrentDate                                                                                │
│ calendarIcon   │ calendar                                                                                       │
│ format         │ DD/MM/YYYY                                                                                     │
│ id             │ date                                                                                           │
│ key            │ date                                                                                           │
│ label          │ $gettext("Date")                                                                               │
│ name           │ date                                                                                           │
│ nextIcon       │ angleRight                                                                                     │
│ prevIcon       │ angleLeft                                                                                      │
│ sectionsSchema │ {'day': {'children': ['$day.getDate()', {'children': [{'children': [{'$el': 'div', 'attrs':    │
│                │ {'class': 'formkit-day-highlight'}, 'if': '$attrs._currentDate().year ===                      │
│                │ $day.getFullYear()'}], 'if': '$attrs._currentDate().month === $day.getMonth()'}], 'if':        │
│                │ '$attrs._currentDate().day === $day.getDate()'}]}}                                             │
└────────────────┴────────────────────────────────────────────────────────────────────────────────────────────────┘

In [55]:
# Does it serialize to postgres? Let's see!
# from_pydantic will return a list of nodes
nodes = list(FormKitSchemaNode.from_pydantic(sf_41_node))

[00:47:06] Creating  11bcf0a8-9873-4c04-a859-aacacdffbb3d                                             ]8;id=324995;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=613562;file:///app/src/formkit_ninja/models.py#570\570]8;;\

           Yielding: Node: date                                                                       ]8;id=601336;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=958333;file:///app/src/formkit_ninja/models.py#592\592]8;;\

In [56]:
for n in nodes:
    kvtable(n.get_node_values())
    modeltable(n.get_node())

# Note that there is one difference on postgres
# Because `sectionsSchema` is something we don't
# have in the postgres schema, it's held within the `additional_props` field

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key              ┃ Value                                                                                        ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit         │ datepicker                                                                                   │
│ additional_props │ {'sectionsSchema': {'day': {'children': ['$day.getDate()', {'children': [{'children':        │
│                  │ [{'$el': 'div', 'attrs': {'class': 'formkit-day-highlight'}, 'if':                           │
│                  │ '$attrs._currentDate().year === $day.getFullYear()'}], 'if': '$attrs._currentDate().month    │
│                  │ === $day.getMonth()'}], 'if': '$attrs._currentDate().day === $day.getDate()'}]}},            │
│                  │ '_currentDate': '$getCurrentDate'}                                                           │
│ calendarIcon     │ calendar                                                                                     │
│ format           │ DD/MM/YYYY                                                                                   │
│ id               │ date                                                                                         │
│ key              │ date                                                                                         │
│ label            │ $gettext("Date")                                                                             │
│ name             │ date                                                                                         │
│ nextIcon         │ angleRight                                                                                   │
│ prevIcon         │ angleLeft                                                                                    │
└──────────────────┴──────────────────────────────────────────────────────────────────────────────────────────────┘

                                                  DatePickerNode                                                   
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key            ┃ Value                                                                                          ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit       │ datepicker                                                                                     │
│ _currentDate   │ $getCurrentDate                                                                                │
│ calendarIcon   │ calendar                                                                                       │
│ format         │ DD/MM/YYYY                                                                                     │
│ id             │ date                                                                                           │
│ key            │ date                                                                                           │
│ label          │ $gettext("Date")                                                                               │
│ name           │ date                                                                                           │
│ nextIcon       │ angleRight                                                                                     │
│ prevIcon       │ angleLeft                                                                                      │
│ sectionsSchema │ {'day': {'children': ['$day.getDate()', {'children': [{'children': [{'$el': 'div', 'attrs':    │
│                │ {'class': 'formkit-day-highlight'}, 'if': '$attrs._currentDate().year ===                      │
│                │ $day.getFullYear()'}], 'if': '$attrs._currentDate().month === $day.getMonth()'}], 'if':        │
│                │ '$attrs._currentDate().day === $day.getDate()'}]}}                                             │
└────────────────┴────────────────────────────────────────────────────────────────────────────────────────────────┘

In [57]:
# Schemas
# Usually one input is not enough, we need a schema to hold multiple inputs
# This is a JSON object not real javascript
schema = '''
[
  {
    $el: 'h1',
    children: 'Register',
    attrs: {
      class: 'text-2xl font-bold mb-4',
    },
  },
  {
    $formkit: 'text',
    name: 'email',
    label: 'Email',
    help: 'This will be used for your account.',
    validation: 'required|email',
  },
  {
    $formkit: 'password',
    name: 'password',
    label: 'Password',
    help: 'Enter your new password.',
    validation: 'required|length:5,16',
  },
  {
    $formkit: 'password',
    name: 'password_confirm',
    label: 'Confirm password',
    help: 'Enter your new password again to confirm it.',
    validation: 'required|confirm',
    validationLabel: 'password confirmation',
  },
  {
    $cmp: 'FormKit',
    props: {
      name: 'eu_citizen',
      type: 'checkbox',
      id: 'eu',
      label: 'Are you a european citizen?',
    },
  },
  {
    $formkit: 'select',
    if: '$get(eu).value', // 👀 Oooo, conditionals!
    name: 'cookie_notice',
    label: 'Cookie notice frequency',
    options: {
      refresh: 'Every page load',
      hourly: 'Ever hour',
      daily: 'Every day',
    },
    help: 'How often should we display a cookie notice?',
  },
]
'''

import json5
schema_as_js = json5.loads(schema)

In [58]:
# So, what happens when we try to parse this?
# It's a list of FormKitNodes
for n in schema_as_js:
    kvtable(n)

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key      ┃ Value                                ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $el      │ h1                                   │
│ attrs    │ {'class': 'text-2xl font-bold mb-4'} │
│ children │ Register                             │
└──────────┴──────────────────────────────────────┘

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key        ┃ Value                               ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit   │ text                                │
│ help       │ This will be used for your account. │
│ label      │ Email                               │
│ name       │ email                               │
│ validation │ required|email                      │
└────────────┴─────────────────────────────────────┘

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key        ┃ Value                    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit   │ password                 │
│ help       │ Enter your new password. │
│ label      │ Password                 │
│ name       │ password                 │
│ validation │ required|length:5,16     │
└────────────┴──────────────────────────┘

┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key             ┃ Value                                        ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit        │ password                                     │
│ help            │ Enter your new password again to confirm it. │
│ label           │ Confirm password                             │
│ name            │ password_confirm                             │
│ validation      │ required|confirm                             │
│ validationLabel │ password confirmation                        │
└─────────────────┴──────────────────────────────────────────────┘

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key   ┃ Value                                                                                          ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $cmp  │ FormKit                                                                                        │
│ props │ {'name': 'eu_citizen', 'type': 'checkbox', 'id': 'eu', 'label': 'Are you a european citizen?'} │
└───────┴────────────────────────────────────────────────────────────────────────────────────────────────┘

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key      ┃ Value                                                                       ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit │ select                                                                      │
│ help     │ How often should we display a cookie notice?                                │
│ if       │ $get(eu).value                                                              │
│ label    │ Cookie notice frequency                                                     │
│ name     │ cookie_notice                                                               │
│ options  │ {'refresh': 'Every page load', 'hourly': 'Ever hour', 'daily': 'Every day'} │
└──────────┴─────────────────────────────────────────────────────────────────────────────┘

In [59]:
from formkit_ninja.formkit_schema import DiscriminatedNodeTypeSchema

# Our "schema"

schema_as_pydantic = DiscriminatedNodeTypeSchema(schema_as_js)

nodes = list(FormKitSchemaNode.from_pydantic(schema_as_pydantic))
print(nodes)

           Creating  d74419ea-2042-4136-82b6-04fb216c7a46                                             ]8;id=65698;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=950798;file:///app/src/formkit_ninja/models.py#570\570]8;;\

           Yielding: $el d74419ea-2042-4136-82b6-04fb216c7a46                                         ]8;id=220176;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=937290;file:///app/src/formkit_ninja/models.py#592\592]8;;\

               Node: Register                                                                         ]8;id=89568;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=82023;file:///app/src/formkit_ninja/models.py#637\637]8;;\

           Creating  d1df73aa-258b-481d-8bf5-7b7ea30b1fed                                             ]8;id=162869;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=977041;file:///app/src/formkit_ninja/models.py#570\570]8;;\

           Yielding: Node: email                                                                      ]8;id=515803;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=215609;file:///app/src/formkit_ninja/models.py#592\592]8;;\

...

           Creating  ebdccfbe-ee98-4f9d-83b4-27a628c93d0c                                             ]8;id=77849;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=957915;file:///app/src/formkit_ninja/models.py#570\570]8;;\

           Yielding: $cmp ebdccfbe-ee98-4f9d-83b4-27a628c93d0c                                        ]8;id=738520;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=660798;file:///app/src/formkit_ninja/models.py#592\592]8;;\

           Creating  340caef8-cd95-485c-9b00-eea924b6a66e                                             ]8;id=915073;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=216896;file:///app/src/formkit_ninja/models.py#570\570]8;;\

           Yielding: Node: cookie_notice                                                              ]8;id=451107;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=502222;file:///app/src/formkit_ninja/models.py#592\592]8;;\

[
    <FormKitSchemaNode: $el d74419ea-2042-4136-82b6-04fb216c7a46>,
    <FormKitSchemaNode: Node: email>,
    <FormKitSchemaNode: Node: password>,
    <FormKitSchemaNode: Node: password_confirm>,
    <FormKitSchemaNode: $cmp ebdccfbe-ee98-4f9d-83b4-27a628c93d0c>,
    <FormKitSchemaNode: Node: cookie_notice>
]

In [60]:
nodes

[<FormKitSchemaNode: $el d74419ea-2042-4136-82b6-04fb216c7a46>,
 <FormKitSchemaNode: Node: email>,
 <FormKitSchemaNode: Node: password>,
 <FormKitSchemaNode: Node: password_confirm>,
 <FormKitSchemaNode: $cmp ebdccfbe-ee98-4f9d-83b4-27a628c93d0c>,
 <FormKitSchemaNode: Node: cookie_notice>]

In [61]:
manytable(schema_as_js)
manytable([n.get_node_values() for n in nodes])
manynodetable([n.get_node() for n in nodes])

┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Index           ┃ 0             ┃ 1             ┃ 2             ┃ 3             ┃ 4             ┃ 5             ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ $cmp            │               │               │               │               │ FormKit       │               │
│ $el             │ h1            │               │               │               │               │               │
│ $formkit        │               │ text          │ password      │ password      │               │ select        │
│ attrs           │ {'class':     │               │               │               │               │               │
│                 │ 'text-2xl     │               │               │               │               │               │
│                 │ font-bold     │               │               │               │               │               │
│                 │ mb-4'}        │               │               │               │               │               │
│ children        │ Register      │               │               │               │               │               │
│ help            │               │ This will be  │ Enter your    │ Enter your    │               │ How often     │
│                 │               │ used for your │ new password. │ new password  │               │ should we     │
│                 │               │ account.      │               │ again to      │               │ display a     │
│                 │               │               │               │ confirm it.   │               │ cookie        │
│                 │               │               │               │               │               │ notice?       │
│ if              │               │               │               │               │               │ $get(eu).val… │
│ label           │               │ Email         │ Password      │ Confirm       │               │ Cookie notice │
│                 │               │               │               │ password      │               │ frequency     │
│ name            │               │ email         │ password      │ password_con… │               │ cookie_notice │
│ options         │               │               │               │               │               │ {'refresh':   │
│                 │               │               │               │               │               │ 'Every page   │
│                 │               │               │               │               │               │ load',        │
│                 │               │               │               │               │               │ 'hourly':     │
│                 │               │               │               │               │               │ 'Ever hour',  │
│                 │               │               │               │               │               │ 'daily':      │
│                 │               │               │               │               │               │ 'Every day'}  │
│ props           │               │               │               │               │ {'name':      │               │
│                 │               │               │               │               │ 'eu_citizen', │               │
│                 │               │               │               │               │ 'type':       │               │
│                 │               │               │               │               │ 'checkbox',   │               │
│                 │               │               │               │               │ 'id': 'eu',   │               │
│                 │               │               │               │               │ 'label': 'Are │               │
│                 │               │               │               │               │ you a         │               │
│                 │               │               │     

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Index            ┃ 0             ┃ 1             ┃ 2             ┃ 3             ┃ 4            ┃ 5             ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ $cmp             │               │               │               │               │ FormKit      │               │
│ $el              │ h1            │               │               │               │              │               │
│ $formkit         │               │ text          │ password      │ password      │              │ select        │
│ attrs            │ {'class':     │               │               │               │              │               │
│                  │ 'text-2xl     │               │               │               │              │               │
│                  │ font-bold     │               │               │               │              │               │
│                  │ mb-4'}        │               │               │               │              │               │
│ children         │ ['Register']  │               │               │               │              │               │
│ help             │               │ This will be  │ Enter your    │ Enter your    │              │ How often     │
│                  │               │ used for your │ new password. │ new password  │              │ should we     │
│                  │               │ account.      │               │ again to      │              │ display a     │
│                  │               │               │               │ confirm it.   │              │ cookie        │
│                  │               │               │               │               │              │ notice?       │
│ if               │               │               │               │               │              │ $get(eu).val… │
│ label            │               │ Email         │ Password      │ Confirm       │              │ Cookie notice │
│                  │               │               │               │ password      │              │ frequency     │
│ name             │               │ email         │ password      │ password_con… │              │ cookie_notice │
│ options          │               │               │               │               │              │ [{'value':    │
│                  │               │               │               │               │              │ 'refresh',    │
│                  │               │               │               │               │              │ 'label':      │
│                  │               │               │               │               │              │ 'refresh'},   │
│                  │               │               │               │               │              │ {'value':     │
│                  │               │               │               │               │              │ 'hourly',     │
│                  │               │               │               │               │              │ 'label':      │
│                  │               │               │               │               │              │ 'hourly'},    │
│                  │               │               │               │               │              │ {'value':     │
│                  │               │               │               │               │              │ 'daily',      │
│                  │               │               │               │               │              │ 'label':      │
│                  │               │               │               │               │              │ 'daily'}]     │
│ props            │               │               │               │               │ {'name':     │               │
│                  │               │               │               │               │ 'eu_citizen… │               │
│                  │               │               │    

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Index            ┃ 0             ┃ 1             ┃ 2             ┃ 3             ┃ 4            ┃ 5             ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ $cmp             │               │               │               │               │ FormKit      │               │
│ $el              │ h1            │               │               │               │              │               │
│ $formkit         │               │ text          │ password      │ password      │              │ select        │
│ attrs            │ {'class':     │               │               │               │              │               │
│                  │ 'text-2xl     │               │               │               │              │               │
│                  │ font-bold     │               │               │               │              │               │
│                  │ mb-4'}        │               │               │               │              │               │
│ help             │               │ This will be  │ Enter your    │ Enter your    │              │ How often     │
│                  │               │ used for your │ new password. │ new password  │              │ should we     │
│                  │               │ account.      │               │ again to      │              │ display a     │
│                  │               │               │               │ confirm it.   │              │ cookie        │
│                  │               │               │               │               │              │ notice?       │
│ if               │               │               │               │               │              │ $get(eu).val… │
│ label            │               │ Email         │ Password      │ Confirm       │              │ Cookie notice │
│                  │               │               │               │ password      │              │ frequency     │
│ name             │               │ email         │ password      │ password_con… │              │ cookie_notice │
│ props            │               │               │               │               │ {'name':     │               │
│                  │               │               │               │               │ 'eu_citizen… │               │
│                  │               │               │               │               │ 'type':      │               │
│                  │               │               │               │               │ 'checkbox',  │               │
│                  │               │               │               │               │ 'id': 'eu',  │               │
│                  │               │               │               │               │ 'label':     │               │
│                  │               │               │               │               │ 'Are you a   │               │
│                  │               │               │               │               │ european     │               │
│                  │               │               │               │               │ citizen?'}   │               │
│ validation       │               │ required|ema… │ required|len… │ required|con… │              │               │
│ validation-label │               │               │               │ password      │              │               │
│                  │               │               │               │ confirmation  │              │               │
└──────────────────┴───────────────┴───────────────┴───────────────┴───────────────┴──────────────┴───────────────┘

In [62]:
# nodes[3].get_node_values()
schema_as_js[3]

from formkit_ninja.formkit_schema import PasswordNode
PasswordNode(**schema_as_js[3])

schema_as_js[3]["validation-label"] = schema_as_js[3].pop("validationLabel")
PasswordNode(**schema_as_js[3])

PasswordNode(children=None, key=None, if_condition=None, for_loop=None, bind=None, meta=None, id=None, name='password_confirm', label='Confirm password', help='Enter your new password again to confirm it.', validation='required|confirm', validationLabel='password confirmation', validationVisibility=None, validationMessages=None, placeholder=None, value=None, prefixIcon=None, classes=None, additional_props=None, node_type='formkit', readonly=None, formkit='password')